In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16913047149982735672
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2431333623086473271
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [3]:
!cat /proc/meminfo

MemTotal:       13294264 kB
MemFree:         7259332 kB
MemAvailable:   11171336 kB
Buffers:          326516 kB
Cached:          3786972 kB
SwapCached:            0 kB
Active:           666648 kB
Inactive:        4933048 kB
Active(anon):       1068 kB
Inactive(anon):  1496760 kB
Active(file):     665580 kB
Inactive(file):  3436288 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              9692 kB
Writeback:             0 kB
AnonPages:       1485404 kB
Mapped:           758416 kB
Shmem:             11620 kB
KReclaimable:     102184 kB
Slab:             148228 kB
SReclaimable:     102184 kB
SUnreclaim:        46044 kB
KernelStack:        4816 kB
PageTables:        16800 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6647132 kB
Committed_AS:    3209468 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       77496 kB
VmallocChunk:          0 kB
Percpu:          

In [58]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 21.7 MB/s eta 0:00:00


In [28]:
from google.colab import files
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import string
import nltk
import math
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

In [5]:
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv.zip to training.1600000.processed.noemoticon.csv.zip


Since the Data size was very large for easy and quick computing that data is downloaded as a.zip file and then unzipped after it it obtained into remote from local.

In [7]:
!unzip training.1600000.processed.noemoticon.csv.zip

Archive:  training.1600000.processed.noemoticon.csv.zip
  inflating: training.1600000.processed.noemoticon.csv  


The data in remote is viewed.

In [8]:
!ls

sample_data
training.1600000.processed.noemoticon.csv
training.1600000.processed.noemoticon.csv.zip


In [9]:
data = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, encoding='ISO-8859-1',names=['target', 'id', 'date', 'flag', 'user', 'text'])

**Viewing the data:**

In [10]:
data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [11]:
data.shape

(1600000, 6)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [14]:
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Data Pre-processing:**

In [15]:
def preprocess_tweet(text):
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Remove mentions
    text = re.sub(r'#', '', text) # Remove hashtags
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in text.split() if word.lower() not in stop_words]) # Tokenize, lemmatize, and remove stop words
    return text

In [22]:
data['text'] = data['text'].apply(preprocess_tweet)

**Visualization of the cleaned data:**

In [23]:
data['text']

0          awww thats bummer shoulda got david carr third...
1          upset cant update facebook texting might cry r...
2          dived many time ball managed save 50 rest go b...
3                            whole body feel itchy like fire
4                                   behaving im mad cant see
                                 ...                        
1599995                        woke school best feeling ever
1599996           thewdbcom cool hear old walt interview â«
1599997                       ready mojo makeover ask detail
1599998    happy 38th birthday boo alll time tupac amaru ...
1599999                                 happy charitytuesday
Name: text, Length: 1600000, dtype: object

**Splitting the Test and Train data:**

In [24]:
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [25]:
vectorizer = TfidfVectorizer(max_features=5000)
train_features = vectorizer.fit_transform(train_data['text'])
test_features = vectorizer.transform(test_data['text'])

**ML Model**

In [26]:
# Training the classifier
clf = MultinomialNB()
clf.fit(train_features, train_data['target'])

MultinomialNB()

In [29]:
# Text generation
def generate_tweet(seed_sentence, n=10):
    current_sentence = seed_sentence
    perplexity = 0
    for i in range(n):
        vectorized_sentence = vectorizer.transform([current_sentence])
        prediction = clf.predict(vectorized_sentence)[0]
        if prediction == 0:
            next_word = np.random.choice(train_data[train_data['target'] == 0]['text'])
        else:
            next_word = np.random.choice(train_data[train_data['target'] == 4]['text'])
        current_sentence += ' ' + next_word
        
        # Calculate perplexity
        prob = clf.predict_proba(vectorized_sentence)
        perplexity += math.log(prob[0][prediction])
    
    # Calculate average perplexity
    avg_perplexity = math.exp(-perplexity/n)
    return current_sentence, avg_perplexity

**Seed Sentence and Perplexity score:**

In [54]:
seed_sentence = "well done"
generated_tweet, perplexity = generate_tweet(seed_sentence)
print("Generated tweet: ", generated_tweet)
print("Perplexity score: ", perplexity)

Generated tweet:  well done leaving state 5 hour miss sg amp people really want right lost beer pong meet 7 im sittin 2day da last gatherin da villafane residence parkesburg b4 sell itafter 14 yr livin sad stuck javascript web20 designer web20 design web20 oh hell yes thunder rain mafra yet though ewwwww quarintine sux bad bored yo read last two entry good stuff im ready get road instead work stuff dry humid outside hate humidity make hair frizz give head ache
Perplexity score:  1.2049839691955315


Array of sentences that contains the Seed sentence "well done"

In [55]:
sentences = []
for i in range(0,len(data["id"])):
  if(data['text'][i].find("well done")>=0):
    sentences.append(data['text'][i])
sentences

['kutner mean knew something going happen someone sad lt3 house well done im still sad',
 'asmith well done u get mp4 version pls make available download somewhere u didnt get c',
 'well done wolf getting promoted im upset charlton confirmed playing division 1 next season',
 'knight amp well done wolf bad luck charlton',
 'liked idea behind yeah wasnt well done think horror movie',
 'got cancelled knew probs strike never heard cancellation good well done show',
 'missed f1 damned thing didnt record still didnt miss much well done red bull congrats',
 'simmo well done yesterday glad moseley hartpury win two positive glaws fan',
 'mean idea lol well done im graphic',
 'well done 100 didnt notice got playstation store rb2',
 'fairly uneventfull weekend glad see become twit lol well done reaching 1 mil 1st',
 'first impression tweetie 4 mac fast love quotconversationquot ui well done love group fb plugin tweetdeck',
 'ward21 well done still 3 assignment 1 presentation 3 exam',
 'probs publ

**Rouge Score:**

In [59]:
from torchmetrics.text.rouge import ROUGEScore
rouge = ROUGEScore()
rouge(sentences, seed_sentence)

{'rouge1_fmeasure': tensor(0.2353),
 'rouge1_precision': tensor(0.1333),
 'rouge1_recall': tensor(1.),
 'rouge2_fmeasure': tensor(0.1333),
 'rouge2_precision': tensor(0.0714),
 'rouge2_recall': tensor(1.),
 'rougeL_fmeasure': tensor(0.2353),
 'rougeL_precision': tensor(0.1333),
 'rougeL_recall': tensor(1.),
 'rougeLsum_fmeasure': tensor(0.2353),
 'rougeLsum_precision': tensor(0.1333),
 'rougeLsum_recall': tensor(1.)}